In [1]:
# Possible sources:
"""
https://github.com/topics/vtuber
1. https://github.com/1996scarlet/OpenVtuber
2. https://github.com/1996scarlet/faster-mobile-retinaface 
   (faster version of 1)
3. https://github.com/pkhungurn/talking-head-anime-demo 
   (may be slow and bg needs to be removed)
"""

'\nhttps://github.com/topics/vtuber\n1. https://github.com/1996scarlet/OpenVtuber\n2. https://github.com/1996scarlet/faster-mobile-retinaface \n   (faster version of 1)\n3. https://github.com/pkhungurn/talking-head-anime-demo \n   (may be slow and bg needs to be removed)\n'

In [1]:
from Talking_Head.Talking_Head import Talking_Head
from putting_it_together import blink_loop
%matplotlib inline
import torch
from PIL import Image
import matplotlib.pyplot as plt
import multiprocess
import gradio as gr
import time
import numpy as np
from copy import deepcopy

pygame 2.1.2 (SDL 2.0.18, Python 3.9.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
device = torch.device("cuda:0")

In [3]:
# Load in the image
global img
img = Image.open("test.png")

In [4]:
# Gradio live image loop documentation:
# https://github.com/gradio-app/gradio/pull/2189

def blink_loop():
    global img
    # Initialize the Talking Head class to add movement to images
    img_anim = Talking_Head(torch.device("cuda:0"), 0.6, automatic_EMA=False)

    # Load in the new image
    img_anim.load_new_image(img=deepcopy(img))
    
    # Start the initial timer
    update_timer_start = time.time()

    # Default pose
    img = img_anim.change_pose()
    
    while True:
        # Wait a little to blink again
        if img_anim.cycle_end:
            # time this function so that it doesn't
            # add to the EMA
            pause_time_start = time.time()
            
            # Blink anywhere between 2 and 7 secods with
            # a mean around 5 seconds (avg blink wait time)
            t = np.clip(np.random.normal(5, 1, size=1)[0], 2, 7)

            plt.pause(t)
            img_anim.cycle_end = False
            
            # Get the pause time
            pause_time = pause_time_start-time.time()
            
            # Update the initial time with the pause time
            update_timer_start -= pause_time
        
        # Update the vector
        img_anim.Move_eyes()
        
        # Change the pose
        img = img_anim.change_pose()
        
        # Get the ending time for the loop
        display_time = time.time()-update_timer_start
        
        # Update the EMA
        img_anim.update_EMA(display_time)
        
        # Start the timer to see how long it takes
        # to display the image and generate a new one
        update_timer_start = time.time()
        
        yield img

In [5]:
def fake_diffusion(steps):
    for i in range(steps):
        time.sleep(1)
        yield np.random.random((200, 200, 3))

io = gr.Interface(blink_loop, inputs=[], outputs=[gr.Image().style(height=256)])
io.queue().launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
